**Webscraping and Cleaning of Airtraffic Data**

First import the necessary Datasets for the notebook

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime, timedelta

**Try to download the content of one website**


page_to_scrape = requests.get('https://www.dfld.de/Mess/StatAirportTag.php?R=601&D=12.07.2024')
soup = BeautifulSoup(page_to_scrape.text, 'html.parser')

entries = soup.findAll("tr", attrs={"class": "5af0f0 trennerlinie"})

**Now change the different Website domains to get the code for all the days since the 01-01-2022**

In [ ]:


# Step 1: Funktion zur Erstellung einer Liste von Datumswerten
def generate_dates(start_date, end_date):
    """Erstellt eine Liste von Datumswerten zwischen zwei Daten."""
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%d.%m.%Y"))
        current_date += timedelta(days=1)
    return date_list

# Step 2: Scraping-Funktion für eine einzelne Tabelle
def scrape_table_for_date(date):
    """Scrapt die Tabelle für ein bestimmtes Datum."""
    # URL mit dem dynamischen Datum
    url = f"https://www.dfld.de/Mess/StatAirportTag.php?R=601&D={date}"
    print(f"Scraping data for date: {date}")
    
    response = requests.get(url)
    response.raise_for_status()  # Sicherstellen, dass die Anfrage erfolgreich war
    
    # HTML-Inhalt mit BeautifulSoup parsen
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Tabelle finden
    table = soup.find("table", class_="table_lines")  # Ersetzen, falls sich die Tabelle ändert
    if not table:
        print(f"No table found for date: {date}")
        return None  # Kein Inhalt für dieses Datum
    
    # Tabellenkopf scrapen
    headers = [header.text.strip() for header in table.find_all("th")]
    if not headers:
        print(f"No headers found for date: {date}")
        return None
    
    # Tabelleninhalte scrapen
    rows = []
    for row in table.find_all("tr"):
        cells = row.find_all("td")
    # Keep rows even if they have fewer cells
        if cells:
            # Fill missing cells with None
            row_data = [cell.text.strip() for cell in cells]
            while len(row_data) < len(headers):
                row_data.append(None)  # Or use "" if you prefer empty strings
            rows.append(row_data)

    
    # DataFrame erstellen
    if rows:
        df = pd.DataFrame(rows, columns=headers)
        return df
    else:
        print(f"No valid data found for date: {date}")
        return None





# Step 3: Daten für mehrere Tage sammeln
def scrape_multiple_days(start_date, end_date, output_file):
    """Scrapt Tabellen für ein Datumsspektrum und speichert sie in einer CSV-Datei."""
    all_data = []  # Liste für die gesammelten Daten
    
    # Liste von Datumswerten erstellen
    dates = generate_dates(start_date, end_date)
    
    # Für jedes Datum scrapen
    for date in dates:
        df = scrape_table_for_date(date)
        if df is not None:
            # Spalte für Datum hinzufügen
            df["Datum"] = date
            all_data.append(df)
    
    # Alle Daten zusammenfügen und in eine CSV-Datei speichern
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv(output_file, index=False)
        print(f"Data successfully saved to {output_file}")
    else:
        print("No data was collected.")

# Step 4: Start- und Enddatum definieren
start_date = datetime.strptime("01.01.2022", "%d.%m.%Y")  # Startdatum
end_date = datetime.strptime("22.11.2024", "%d.%m.%Y")    # Enddatum

# Daten scrapen und exportieren
output_file = "airport_statistics.csv"
scrape_multiple_days(start_date, end_date, output_file)



Scraping data for date: 01.01.2022
Scraping data for date: 02.01.2022
Scraping data for date: 03.01.2022
Scraping data for date: 04.01.2022
Scraping data for date: 05.01.2022
Scraping data for date: 06.01.2022
No table found for date: 06.01.2022
Scraping data for date: 07.01.2022
Scraping data for date: 08.01.2022
Scraping data for date: 09.01.2022
Scraping data for date: 10.01.2022
Scraping data for date: 11.01.2022
Scraping data for date: 12.01.2022
Scraping data for date: 13.01.2022
Scraping data for date: 14.01.2022
Scraping data for date: 15.01.2022
Scraping data for date: 16.01.2022
Scraping data for date: 17.01.2022
Scraping data for date: 18.01.2022
Scraping data for date: 19.01.2022
Scraping data for date: 20.01.2022
Scraping data for date: 21.01.2022
Scraping data for date: 22.01.2022
Scraping data for date: 23.01.2022
Scraping data for date: 24.01.2022
Scraping data for date: 25.01.2022
Scraping data for date: 26.01.2022
Scraping data for date: 27.01.2022
Scraping data for d

**Cleaning Part** 
Steps:
1. Import Data and see what I am working with
2. get the flights allocated to the hours
3. fill up the days, where no flights were recorded, because its unlikely 
4. ensure the date is still in the right format 

In [3]:
airport_statistics = pd.read_csv("../../00_Uncleaned_Data/Features/04airport_statistics.csv")
airport_statistics.head()

,Zeit,Dep/Arr,Fluggesellschaft,Dep-Arr,Registrierung Callsign / Flugnummer,Flugzeug,Klasse,Piste,Datum
0,00:13:33,,NaN,NaN,HB-ZQGRGA02,EC45,L,33.0,01.01.2022
1,06:22:39,,DS (CH)easyJet Switzerland,NaN,EZS5632,NaN,NaN,33.0,01.01.2022
2,06:26:36,,LH (DE)Deutsche Lufthansa,NaN,D-ACNGDLH6TN,CRJ9,M,33.0,01.01.2022
3,06:29:23,,KL (NL)KLM Royal Dutch Airlines,NaN,PH-EXRKLM84X,E75L,M,33.0,01.01.2022
4,06:37:38,,8C (CN)Shanxi Airlines,NaN,CXI7265,NaN,NaN,15.0,01.01.2022


In [4]:
# Assuming airport_statistics is a pandas DataFrame

# Filter out invalid 'Zeit' values before conversion
airport_statistics = airport_statistics[~airport_statistics["Zeit"].str.contains("Summe")]

# Convert the 'Datum' column to a proper datetime format
airport_statistics["Datum"] = pd.to_datetime(airport_statistics["Datum"], format="%d.%m.%Y", utc=True)

# Convert the 'Zeit' column to a time format
airport_statistics["Zeit"] = pd.to_datetime(airport_statistics["Zeit"], format="%H:%M:%S", utc=True).dt.time

# Create a new column for the hour extracted from 'Zeit'
airport_statistics["Hour"] = pd.to_datetime(airport_statistics["Zeit"].astype(str), format="%H:%M:%S").dt.hour

# Group by 'Datum' and 'Hour' to calculate the hourly count of traffic
hourly_sum = airport_statistics.groupby(["Datum", "Hour"]).size().reset_index(name="Traffic")

# Ensure all hours (0-23) are present for each day
all_dates = pd.date_range(hourly_sum["Datum"].min(), hourly_sum["Datum"].max(), freq="D")
all_hours = pd.DataFrame({"Hour": range(24)})
all_combinations = pd.MultiIndex.from_product([all_dates, all_hours["Hour"]], names=["Datum", "Hour"])

hourly_sum = hourly_sum.set_index(["Datum", "Hour"]).reindex(all_combinations, fill_value=0).reset_index()

# Add the weekday for each row
hourly_sum["Weekday"] = hourly_sum["Datum"].dt.day_name()

# Calculate hourly averages for each weekday
hourly_weekday_averages = hourly_sum.groupby(["Weekday", "Hour"])["Traffic"].mean().reset_index()

# Ensure the 'Traffic' column is of a compatible dtype (float)
hourly_sum["Traffic"] = hourly_sum["Traffic"].astype(float)

# Fill in missing days with hourly averages for the same weekday
missing_days = hourly_sum.groupby("Datum")["Traffic"].sum() == 0  # Identify missing days
for missing_date in hourly_sum["Datum"].unique()[missing_days]:
    weekday = missing_date.day_name()
    avg_values = hourly_weekday_averages[hourly_weekday_averages["Weekday"] == weekday]
    for _, row in avg_values.iterrows():
        hourly_sum.loc[
            (hourly_sum["Datum"] == missing_date) & (hourly_sum["Hour"] == row["Hour"]), "Traffic"
        ] = row["Traffic"]

# Group by 'Datum' to calculate the daily count of traffic
daily_sum = hourly_sum.groupby("Datum").agg(Traffic=("Traffic", "sum")).reset_index()


total_sum = daily_sum["Traffic"].sum()

# Display the results
print("Hourly Traffic Data (missing hours filled with weekday averages):")
print(hourly_sum)



print("\nDaily Traffic Data (missing days filled with hourly averages of weekday):")
print(daily_sum)

print(total_sum)


Hourly Traffic Data (missing hours filled with weekday averages):
                          Datum  Hour  Traffic   Weekday
0     2022-01-01 00:00:00+00:00     0      1.0  Saturday
1     2022-01-01 00:00:00+00:00     1      0.0  Saturday
2     2022-01-01 00:00:00+00:00     2      0.0  Saturday
3     2022-01-01 00:00:00+00:00     3      0.0  Saturday
4     2022-01-01 00:00:00+00:00     4      0.0  Saturday
...                         ...   ...      ...       ...
25363 2024-11-22 00:00:00+00:00    19      9.0    Friday
25364 2024-11-22 00:00:00+00:00    20     11.0    Friday
25365 2024-11-22 00:00:00+00:00    21     12.0    Friday
25366 2024-11-22 00:00:00+00:00    22      9.0    Friday
25367 2024-11-22 00:00:00+00:00    23      5.0    Friday

[25368 rows x 4 columns]

Daily Traffic Data (missing days filled with hourly averages of weekday):
                         Datum  Traffic
0    2022-01-01 00:00:00+00:00    168.0
1    2022-01-02 00:00:00+00:00    191.0
2    2022-01-03 00:00:00+00:0

The code doesnt want to keep the hour somehow, so its entered afterwards via adding hour column to the date

In [9]:
# Combine 'Datum' and 'Hour' into a single datetime column
hourly_sum["Datum"] = hourly_sum.apply(
    lambda row: pd.Timestamp(row["Datum"]) + pd.Timedelta(hours=row["Hour"]), axis=1
)

# Ensure the new column is in the desired datetime format
hourly_sum["Datum"] = pd.to_datetime(hourly_sum["Datum"], utc=True)

In [11]:
airport_statistics.head()
hourly_sum.head(25)
hourly_sum.drop(columns=["Weekday"], inplace=True)


Save the cleaned Data to the cleaned Data Folder


In [ ]:

hourly_sum.to_csv("../../02_Cleaned/Features/04Airport_traffic_hourly_cleaned.csv")